# 01 - Data Preparation (Local Database)

This notebook guides you through creating and populating your local PostgreSQL database using the project scripts:

- `src/database/connection.py` (create/init DB + schema)
- `src/database/populate.py` (load MS MARCO and insert data)

At the end, you will verify that the tables were populated.

## Prerequisites

Before running, ensure:

1. PostgreSQL is running locally.
2. You have database credentials ready.
3. You created a `.env` file at the project root (optional, defaults exist).

Expected environment variables:

- `DB_HOST` (default: `localhost`)
- `DB_PORT` (default: `5432`)
- `DB_NAME` (default: `msmarco_db`)
- `DB_USER` (default: `postgres`)
- `DB_PASSWORD` (default: `postgres`)
- `HF_TOKEN` (optional, for Hugging Face access)

In [ ]:
from pathlib import Path

# Resolve project root from notebook location
project_root = Path.cwd().resolve().parent
print(f"Project root: {project_root}")

env_path = project_root / ".env"
if env_path.exists():
    print(f"Found .env file: {env_path}")
else:
    print("No .env found at project root. Default values from config.py will be used.")
    print("You can create one manually if needed.")

## 1) Install Python dependencies

Run this once per environment.

In [ ]:
import sys
import subprocess

requirements_file = project_root / "requirements.txt"
print(f"Installing dependencies from: {requirements_file}")

subprocess.run(
    [sys.executable, "-m", "pip", "install", "-r", str(requirements_file)],
    check=True,
)

## 2) Create / initialize database schema

This step uses `connection.py` to:

- create the target database if it does not exist
- execute `schema.sql`

In [ ]:
import sys
import subprocess

print("Running: python -m src.database.connection")

subprocess.run([sys.executable, "-m", "src.database.connection"], check=True, cwd=str(project_root))

## 3) Populate database from MS MARCO

This step can take some time and insert a large volume of rows.

The script will:

- download/load the dataset (`microsoft/ms_marco`, `v1.1`)
- extract rows
- insert `queries`, `passages`, and `qrels`

In [ ]:
import sys
import subprocess

module_name = "src.database.populate"
print(f"Running: python -m {module_name}")

subprocess.run([sys.executable, "-m", module_name], check=True, cwd=str(project_root))

## 4) Verify database population

This step display the first 10 rows of the three core tables to confirm successful insertion.

In [41]:
import pandas as pd
import warnings
from src.database.connection import get_connection

# Suppress pandas DBAPI warning for psycopg2 connections
warnings.filterwarnings(
    "ignore",
    message="pandas only supports SQLAlchemy connectable.*",
    category=UserWarning,
)

conn = get_connection()

try:
    tables = ["queries", "passages", "qrels"]

    print("\n=== First 10 rows from queries ===")
    display(pd.read_sql_query("SELECT * FROM queries ORDER BY id LIMIT 10", conn))

    print("\n=== First 10 rows from passages ===")
    display(pd.read_sql_query("SELECT * FROM passages ORDER BY id LIMIT 10", conn))

    print("\n=== First 10 rows from qrels ===")
    display(pd.read_sql_query("SELECT * FROM qrels ORDER BY id LIMIT 10", conn))
finally:
    conn.close()

2026-02-21 00:31:55,845 - INFO - Database connection established successfully.



=== First 10 rows from queries ===


,id,text
0,19699,what is rba
1,19700,was ronald reagan a democrat
2,19701,how long do you need for sydney and surroundin...
3,19702,price to install tile in shower
4,19703,why conversion observed in body
5,19704,where are the lungs located in the back
6,19705,cost to get a patent
7,19706,what does a metabolic acidosis need to reverse...
8,19707,best tragedies of ancient greece
9,19708,what is a conifer



=== First 10 rows from passages ===


,id,text,title
0,1,"Since 2007, the RBA's outstanding reputation h...",None
1,2,The Reserve Bank of Australia (RBA) came into ...,None
2,3,RBA Recognized with the 2014 Microsoft US Regi...,None
3,4,The inner workings of a rebuildable atomizer a...,None
4,5,Results-Based Accountability® (also known as R...,None
5,6,Results-Based Accountability® (also known as R...,None
6,7,"RBA uses a data-driven, decision-making proces...",None
7,8,vs. NetIQ Identity Manager. Risk-based authent...,None
8,9,"A rebuildable atomizer (RBA), often referred t...",None
9,10,Get To Know Us. RBA is a digital and technolog...,None



=== First 10 rows from qrels ===


,id,query_id,passage_id,relevance
0,1,19699,1,0
1,2,19699,2,0
2,3,19699,3,0
3,4,19699,4,0
4,5,19699,5,0
5,6,19699,6,1
6,7,19699,7,0
7,8,19699,8,0
8,9,19699,9,0
9,10,19699,10,0


## Notes

- If population fails due to connectivity/authentication, re-check your PostgreSQL credentials in `.env`.
- If Hugging Face access is restricted in your environment, set `HF_TOKEN` in `.env`.
- You can re-run the initialization safely as table creation uses your SQL schema definition.